# Clase 23: Despliegue

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

- Conocer como exportar modelos de ML usando `pickle`
- Aprender a montar ambientes virtuales con `conda`
- Interiorizar al estudiante sobre aplicaciones web y su división en Front End y Back End
- Levantar una aplicación web usando `FastAPI` y `Gradio`

## Objetivo

Esta clase tiene como objetivo introducir a los estudiantes a algunas herramientas para desplegar modelos de ML.

## Datos de esta clase

Para esta clase, ejemplificaremos lo aprendido utilizando el clásico **Iris Dataset**. El objetivo es simple: entrenar un modelo de ML para clasificar flores del tipo iris en sus 3 categorías: setosa, versicolor y virginica. Las características disponibles son 4: el largo y ancho del pétalo y sépalo.

![Iris Dataset](../../recursos/2024-01/despliegue/images/iris.png)

Comencemos primero importando los datos. Podemos hacer esto de manera simple usando la API de `sklearn`:

In [ ]:
from sklearn.datasets import load_iris

iris_df = load_iris(as_frame=True) # cargar dataset
X = iris_df["data"] # features para predecir
y = iris_df["target"] # variable target, 0: setosa, 1: versicolor, 2: viginica

# features disponibles
X 

In [ ]:
# variable a predecir
y 

Con los datos ya ingestados, podemos entrenar un clasificador de `RandomForest` de manera simple:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

seed = 3380

# separamos los datos
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3, random_state = seed)

model = RandomForestClassifier(random_state = seed) # instanciar modelo
model.fit(X_train, y_train) # fit

y_pred = model.predict(X_test) # predict sobre X_test
accuracy_score(y_test, y_pred) # performance

Genial! Entrenamos efectivamente un modelo de ML para resolver nuestro problema :)

Del entrenamiento, notamos que nuestro modelo tiene una alta capacidad de predicción, acertando el 97.7% de los casos en el conjunto de test.

## Entrené mi modelo... ¿y ahora qué?

<center>
<img src='https://media1.tenor.com/m/NeOTch1WXawAAAAd/finding-nemo-bags.gif' width=450  />
</center>

Supongamos que tenemos un cercano (jefe, colega, mamá, hermano, perro, etc) al que queramos mostrarle el funcionamiento de nuestro modelo. ¿Qué es lo que tendríamos que hacer para lograr esto?

Una primera idea sería mostrarles directamente nuestro código montado en nuestro equipo y mostrar como se ejecuta... aunque esto dificilmente es una solución aceptable en términos productivos. 

Para atacar este problema de manera efectiva, nos dedicaremos el resto de la clase a aprender diferentes técnicas de **despliegue** de nuestro modelo.



## Exportar modelo

Antes de desplegar el modelo entrenado, el primer paso es **serializar** nuestro modelo para que luego sea ingestado en otro dispositivo.

Podemos lograr esto usando la librería `pickle`:

In [ ]:
import pickle

# crear "model.pkl" con nuestro modelo serializado
with open('./model.pkl', 'wb') as file:
    pickle.dump(model, file)

Podemos validar que nuestro modelo se carga de manera efectiva:

In [ ]:
model

In [ ]:
# borramos modelo
del model

# noten como ya no podemos predecir sobre X pues model ya no existe
model.predict(X)

In [ ]:
# cargamos modelo
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

# verificamos funcionamiento
model.predict(X)

## Esencia del despliegue

Pasemos ahora al despliegue de nuestro modelo. De forma general, cualquier herramienta de despliegue debería ser capaz de:

0. Cargar nuestro modelo 
1. Recibir nuevos datos (e.g: una nueva *fila* de $X$)
2. Pre procesar los datos de manera adecuada (escalar, transformar a one_hot, crear nuevas features, etc)
3. Generar una predicción sobre los datos procesados
4. Retornar la predicción generada

<div style="text-align: center;">
<img src='../../recursos/2024-01/despliegue/images/diagrama_despliegue.png' width = 650/>
</div>

> **Pregunta:** ¿Qué herramienta aprendida en el curso nos puede ayudar a los pasos 2 y 3?

Veamos como se vería esto de manera conceptual en el código:

In [ ]:
# cargar modelo
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

labels_dict = {0: 'setosa', 1: 'versicolor', 2: 'virginica'} # diccionario de etiquetas
def make_prediction(sepal_length: float, sepal_width: float, petal_length: float, petal_width: float):
    '''
    función que devuelve la predicción del modelo dado un set de atributos
    '''

    # mantener el orden!
    features = [
        [sepal_length, sepal_width, petal_length, petal_width] # obs a predecir, OJO con el orden!! 
    ]
    
    prediction = model.predict(features).item() # generar prediccion
    label = labels_dict[prediction] # transformar a etiqueta

    return label # retornar prediccion

make_prediction(sepal_length=2, sepal_width=1, petal_length=1, petal_width=1)

Noten que si queremos cambiar los atributos de entrada, simplemente debemos cambiar los valores de los parámetros de la función:

In [ ]:
make_prediction(sepal_length=5, sepal_width=4, petal_length=5, petal_width=3)

In [ ]:
make_prediction(sepal_length=1, sepal_width=7, petal_length=1.7, petal_width=0.5)

Guardaremos una copia de esta función en `recursos/2024-02/make_prediction.py` para volver a utilizarla más adelante.

## Ambientes virtuales

La idea de esta sección es **generar las condiciones necesarias para que nuestro proyecto pueda ser ejecutado otros dispositivos** de manera fácil. Una primera aproximación para lograr esto es a través de **entornos virtuales**, los cuales pueden ser interpretados como un espacio virtual aislado donde podemos instalar y levantar configuraciones *custom* de nuestro proyecto.

> **Pregunta:** ¿Se les ocurre algún ejemplo donde sea útil hacer uso de estos ambientes?

### ¿Cómo crear nuestro ambiente virtual?

El primer paso para trabajar sobre ambientes virtuales es instalar [anaconda](https://www.anaconda.com). Con la distribución instalada, podemos levantar un ambiente virtual ejecutando en la terminal los siguientes comandos:

```bash
conda create --name nombre_ambiente python=version_python -y # crear ambiente virtual
```

donde *version_python* equivale a la versión de python deseada (3.9, 3.10, 3.11, etc). Luego de crear nuestro ambiente, es necesario entrar a este por medio de:

```bash
conda activate nombre_ambiente # activar ambiente virtual
```

```bash
conda deactivate # desactivar ambiente virtual
```

### Levantando un proyecto en un ambiente virtual

Para levantar un proyecto en un ambiente virtual, necesitaremos de los siguientes elementos:

- **main.py:** Código python a ejecutar (basicamente el código python que programaron en su proyecto)
- **requirements.txt:** Archivo con las liberias necesarias para ejecutar `main.py` (pandas==2.2.2, etc.)
- **.env:** Archivo con las credenciales necesarias para ejecutar el proyecto (API KEYS, etc.)

Noten que este ambiente de python es un ambiente *virgen* y no tiene ninguna librería instalada. Pueden verificar lo anterior por medio de:

```bash
pip freeze # printear librerias instaladas por pip
```

Para instalar las librerias necesarias, lo pueden realizar de manera simple por medio de:

```bash
pip install -r requirements.txt
```

Al contrario, si desean exportar las librerias utilizadas en su proyecto a un archivo `requirements.txt`:

```bash
pip freeze > requirements.txt
```

**Importante**: Noten que usando ambientes virtuales, somos capaces de montar nuestro modelo de manera fácil en otro equipo! Sólo necesitaríamos:
- Modelo (`model.pkl`)
- Librerías (`requirements.txt`)
- Código de ejecución (`make_predictions.py` o `main.py`)

> **Pregunta**: ¿Conocen otra herramienta que pueda resolver el mismo problema?

#### Paréntesis: Exportando librerías de ambientes de `conda`

Al trabajar con ambientes de `conda`, algunas librerias quedan antecedidas por "@", por ejemplo:

```text
zipp @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/zipp_1707348942775/work
```

Esto es un **problema** pues levantará un error para instalar las librerías en un nuevo entorno. Pueden solucionar esto de manera fácil exportando sus librerías a través de:

```bash
pip list --format=freeze > requirements.txt
```

## Aplicación Web

Como ya conocemos lo básico para montar nuestro modelo en otros dispositivos, profundicemos ahora en cómo desplegar nuestro modelo para que terceros puedan interactuar con nuestro trabajo.

La forma más robusta para desplegar una solución basada en machine learning es a través de una **aplicación web**, es decir, una aplicación almacenada en un servidor remoto con la que podamos interactuar a través de un navegador web.

Una aplicación web se compone de 2 elementos principales:
- **Frontend**: Lo que se muestra al usuario.
- **Backend**: Procesamiento "tras bambalinas".

> **Pregunta:** ¿Es posible levantar el back sin el front, o vice versa?

A lo largo de esta sección veremos en detalle cómo implementar cada uno de estos componentes. Manos a la obra!

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*FcJQX2zzna7-rdEocH3jYw.png'/>

### Front End

### ¿Qué es el Front End?

<center>
<img src='https://media1.tenor.com/m/DfXYNBOTEQ8AAAAd/react-fron-end.gif' width=300  />
</center>

En el contexto de aplicaciones web, **Front End se refiere a la parte de la aplicación que interactúa con los usuarios** (conocida también como el *Cliente*). En términos sencillos, es todo lo que vemos en la pantalla cuando accedemos a un sitio web o aplicación, por ejemplo: tipos de letra, colores, formato para diferentes tamaños de pantalla, desplazamientos, efectos visuales, etc.

Algunos *framework* comunes para desarrollar componentes de Front End son:

- `React`
- `Vue`
- `Angular`
- `Gradio`
- `Streamlit`

Si bien la mayoria de estos framework son basados en Javascript, `Gradio` y `Streamlit` utilizan Python para levantar el Cliente (aunque por debajo igual utilizan Javascript). Para esta clase nos enfocaremos en `Gradio`.

Comencemos!

### Gradio

*Esta sección está basada en el siguiente [tutorial](https://www.youtube.com/watch?v=97KxA1r184o)*.

![Gradio](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQdC1xNkDaMNEbtQRyupw5v32HSGVA6w0zNjA&s)

[gradio](https://www.gradio.app) es una librería de Python similar a `Streamlit` que permite generar **demos** de manera simple especificando los **componentes** de entrada y salida esperados por tu modelo de machine learning.

¿A qué nos referimos con componentes de entrada y salida? Gradio viene con diferentes componentes para diferentes tipos de modelos de machine learning. Algunos ejemplos:

*   Para un **clasificador de imágenes**, el input esperado es de tipo `Image` y la salida es del tipo `Label`.
*   Para un modelo de **speech recognition**, el input esperado es del tipo `Microphone` (lo que permite al usuario grabar desde el navegador) o `Audio` (lo que permite a los usuarios subir sus propios archivos de audio), mientras que la salida es del tipo `Text`.
* Para un modelo de **questiong answering**, se esperan dos entradas: [`Text`, `Text`], una entrada de texto para el párrafo y otro texto para la pregunta, mientras que la salida es del tipo `Text` para contener la respuesta generada.

Una lista completa de los componentes habilitados se puede encontrar en la [documentación](https://gradio.app/docs/).

Además de los componentes de entrada y salida, Gradio espera un tercer parámetro: **la función de predicción**. Este parámetro puede ser ***cualquier* función regular de Python** que reciba los parámetros correspondientes a los componentes de entrada y retorne una salida congruente con los componentes de salida.

Veamos esto en código!

In [ ]:
#!pip install gradio --upgrade

In [ ]:
import gradio as gr

demo = gr.Interface(fn = make_prediction, # noten como estamos usando la función que generamos anteriormente
                    inputs = ["number", "number", "number", "number"], # valores de entrada
                    outputs = ["text"]) # valor de salida

demo.launch(share = True) # share = True: nos permite compartir el demo con quien queramos!

Genial! Ahora que tenemos una primera versión, veamos como podemos mejorarla un poco en términos estéticos:

In [ ]:
demo = gr.Interface(
    fn = make_prediction,
    inputs = [ # definimos el intervalo y asignamos un nombre a cada input
        gr.Slider(label = 'Sepal Length', minimum = 0, maximum = 10),
        gr.Slider(label = 'Sepal Width', minimum = 0, maximum = 10), 
        gr.Slider(label = 'Petal Length', minimum = 0, maximum = 10), 
        gr.Slider(label = 'Petal Width', minimum = 0, maximum = 10)],
    outputs = gr.Text(label = 'Predicted Label'), # se define un nombre para la salida
    title = 'Iris ML Demo', # asignar un titulo al demo
    examples=[[0.5, 1.5, 2.5, 3.5], [1, 3, 5, 7]], # generar ejemplos
    )

demo.launch(share = True)

Genial! Revisemos ahora como customizar aun más el *display* de nuestra aplicación mediante [gr.Blocks](https://www.gradio.app/docs/gradio/blocks):

In [ ]:
with gr.Blocks(theme = gr.themes.Base()) as demo:

    # agregamos un markdown para describir la aplicacion
    gr.Markdown(
    """
    # Iris ML Demo
    Bienvenid@ a Iris ML demo! Esta herramienta esta diseñada para predecir la clase de una flor a partir de sus características usando Machine Learning.
    ## Cómo usar este demo?
    Usar esta herramienta es fácil! Sólo debes seguir los siguientes pasos:
    1. Fijar los valores de **Sepal Length**, **Sepal Width**, **Petal Length** y **Petal Width**.
    2. Observar el tipo de flor que predice el modelo.
    
    Eso es todo! Estás list@ para explorar y predecir diferentes tipos de flores. Que lo disfrutes!
    """)

    # definimos explicitamente la posicion de los elementos
    with gr.Row():
        with gr.Column():
            sepal_length_slider = gr.Slider(label = 'Sepal Length', minimum = 0, maximum = 10, value = 3.8)
            sepal_width_slider = gr.Slider(label = 'Sepal Width', minimum = 0, maximum = 10, value = 6.4)
            petal_length_slider = gr.Slider(label = 'Petal Length', minimum = 0, maximum = 10, value = 7.3)
            petal_width_slider = gr.Slider(label = 'Petal Width', minimum = 0, maximum = 10, value = 4.9)

        with gr.Column():
            label = gr.Text(label = 'Predicted Label') # se define un nombre para la salida
    
    with gr.Row():
        button = gr.Button(value = 'Predict!')

    # setear interactividad
    inputs = [sepal_length_slider, sepal_width_slider, petal_length_slider, petal_width_slider]
    outputs = [label]
    button.click(fn = make_prediction, inputs = inputs, outputs = outputs)

    examples = [
        [0.5, 1.5, 2.5, 3.5], # example 1
        [1, 3, 5, 7], # example 2
    ]
    gr.Examples(examples = examples, inputs = inputs) 

    demo.launch(share = True)

Pueden encontrar la totalidad del código en `recursos/2024-02/frontend.py`.

**Bonus:** [Tutorial para Desplegar su aplicación de gradio en un servidor de HuggingFace](https://www.youtube.com/watch?v=97KxA1r184o).

### Backend

#### ¿Qué es el Back End?

<center>
<img src='https://media1.tenor.com/m/NoxXhCo1EU4AAAAC/figura-backend.gif' width=125  />
</center>

En el contexto de aplicaciones web, **Back End se refiere a la parte de la aplicación que opera detrás de escena y gestiona la lógica del servidor, los datos y la integración con otras aplicaciones o servicios**. Es el núcleo que se encarga de procesar solicitudes desde el Front End, acceder a bases de datos, realizar cálculos, y devolver respuestas para que el cliente las muestre de manera amigable.

Algunos ejemplos de procesamientos a realizar en el Back End contemplan:

- **Modelos de ML**: Inferencia de nuevas observaciones usando un modelo de ML.
- **Gestión de Datos:** Operaciones CRUD (crear, leer, actualizar y eliminar datos) en bases de datos.
- **Autenticación y Seguridad:** Verificación de usuarios, gestión de permisos y protección de datos.
- **Integraciones:** Comunicación con servicios externos (por ejemplo, APIs de terceros).

**Lenguajes y Frameworks Comunes:**
- Python: `Django, Flask, FastAPI`
- JavaScript/TypeScript: `Node.js`
- Java: `Spring Boot`
- Ruby: `Ruby on Rails`
- PHP: `Laravel`

En esta clase nos enfocaremos en usar un backend sencillo utilizando `FastAPI` en Python, que nos permite crear `API REST` de manera rápida y eficiente.

### ¿Qué es una API?

Una **API (Application Programming Interface)** es un conjunto de reglas y herramientas que permiten a diferentes sistemas, aplicaciones o servicios comunicarse entre sí. Es como un "contrato" que define cómo se deben realizar las interacciones entre ellos, facilitando el intercambio de datos y funcionalidades. De esta manera, las APIs son el **componente clave que permiten que el Front End y el Back End se trabajen de manera conjunta.**

Características Clave de las APIs:
- **Interoperabilidad:** Permiten que diferentes tecnologías y plataformas trabajen juntas.
- **Estandarización:** Usan protocolos y formatos bien definidos, como HTTP, JSON o XML.
- **Reusabilidad:** Un solo API puede ser utilizado por múltiples aplicaciones o servicios.

### ¿Qué es una API REST?

Una **API REST (Representational State Transfer)** es un tipo específico de API que sigue los principios de arquitectura REST. Este enfoque se basa en el uso de HTTP para manejar las operaciones sobre los datos y los recursos que maneja la aplicación. REST es ampliamente utilizado debido a su simplicidad y capacidad para integrarse fácilmente con aplicaciones web.

Una API REST considera los siguientes principios:
1. **Arquitectura Cliente-Servidor:** El Front End (cliente) y el Backend (servidor) están separados y se comunican mediante peticiones HTTP.
2. **Sin Estado:** Cada solicitud enviada desde el cliente al servidor debe contener toda la información necesaria para procesarla, sin depender de un estado previo.
3. **Interfaz Uniforme:** Los recursos se identifican mediante URLs claras y consistentes.
4. **Operaciones HTTP Estándar:**
     - **GET**: Solicitud para pedir datos.
     - **POST**: Enviar datos (en el cuerpo de la solicitud) comunmente para ser procesados y guardados.
     - **DELETE**: Elimina un dato.
     - **PUT**: Actualiza un dato.
5. **Uso de Representaciones:** Los datos se transfieren usando formatos como JSON o XML.

Por otro lado, es útil tener en cuenta los [códigos de respuesta](https://http.cat/) al interactuar con nuestra API:
  - `1xx` - Respuestas informativas
  - `2xx` - Respuestas satisfactorias 
  - `3xx` - Redirecciones 
  - `4xx` - Errores de los clientes 
  - `5xx` - Errores de los servidores 

De esta manera, el objetivo de esta sección es **utilizar FastAPI para generar una API REST que permita entregar las inferencias de nuestro modelo de ML al cliente.**

#### FastAPI

Ya que conocemos lo que es una API REST, revisemos como podemos usar `FastAPI` para implementar nuestra propia API.

[FastAPI](https://fastapi.tiangolo.com) es un framework moderno y de alto rendimiento para construir APIs con Python, que destaca por su simplicidad, rapidez y eficiencia. FastAPI permite desarrollar aplicaciones web rápidamente utilizando la tipificación de Python para generar documentación automática y validaciones de datos sin esfuerzo adicional. Con una curva de aprendizaje amigable y un rendimiento comparable a frameworks como Node.js y Go, FastAPI es una excelente elección para aprender y dominar el desarrollo de APIs en Python.

Comencemos instalando la libreria:

<img src='../../recursos/2024-01/despliegue/images/fastapi.png'/>

In [ ]:
#pip install "fastapi[all]"

Con la libreria instalada, pasemos ahora a escribir nuestra API!

**Nota importante: A diferencia de la sección de `gradio`, será necesario ejecutar nuestro código usando la terminal. Encontrarán el código completo de la API en `recursos/2024-02/backend.py` (recuerden moverse hacia recursos/2024-02/ para ejecutarlo)**

Para usar `fastapi`, lo primero que se debe realizar es generar una instancia del módulo FastAPI:

In [ ]:
from fastapi import FastAPI

# crear aplicación
app = FastAPI()

Para levantar nuestra aplicación, tenemos 2 opciones:

1. Escribir en la terminal el siguiente comando (opción recomendada):

```python
uvicorn nombre_script:app
```

2. Adjuntar al final del código de la aplicación el siguiente *chunk*:

```python
if __name__ == '__main__':
    uvicorn.run('nombre_script:app')
```

El siguiente paso es crear un *home* o vista default de nuestra aplicación. Usualmente, esta vista está hecha para introducir al usuario al funcionamiento de la aplicación.

Veamos como podemos implementar esto en nuestra aplicación:

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get('/') # ruta
async def home(): 
    return {'Hello': 'World'}

Noten como FastAPI hace uso de *decoradores* para definir las rutas de la aplicación (donde en este caso, estamos definiendo un `GET`).

Luego si levantamos nuestra aplicación e ingresamos a [http://127.0.0.1:8000](http://127.0.0.1:8000), deberiamos obtener como respuesta:

```{python}
{'Hello': 'World'}
```

> **Pregunta:** ¿Qué es un json? ¿Porqué es deseable hacer uso de este tipo en nuestra API?

Veamos ahora como definir una segunda ruta de acceso:

In [ ]:
from fastapi import FastAPI
from make_prediction import make_prediction

# init app
app = FastAPI()

# def home
@app.get('/') # ruta
async def home():
    return {'Hello': 'World'}

@app.get('/classroom') # ruta
async def classroom():
    return {'Message': 'This is my first API :)'}

Donde si ingresan a [http://127.0.0.1:8000/classroom](http://127.0.0.1:8000/classroom) deberian observar:

```python
{'Message': 'This is my first API :)'}
```

Genial! Ahora que ya sabemos como definir algunas rutas de acceso en nuestra aplicación, veamos como implementar un método `POST` para desplegar nuestro modelo.

> **Pregunta:** ¿Porqué es deseable generar un método `POST` para el despliegue de nuestra solución?

In [ ]:
from fastapi import FastAPI

# init app
app = FastAPI()

# def home
@app.get('/') # ruta
async def home():
    return {'Hello': 'World'}

@app.get('/classroom') # ruta
def classroom():
    return {'Message': 'This is my first API :)'}

# def predict method
@app.post("/predict") # ruta
async def predict(sepal_length: float, sepal_width: float, petal_length: float, petal_width: float): # parametros de entrada

    label = make_prediction(sepal_length, sepal_width, petal_length, petal_width) # generar prediccion

    return {"label": label} # retornar prediccion

Como pueden ver, desplegar nuestra solución fue sumamente simple ya que pudimos reciclar gran parte de lo hecho en `make_prediction`. Probemos ahora nuestra API!

> **Pregunta:** ¿Qué deberia pasar si ingresamos a [http://127.0.0.1:8000/predict](http://127.0.0.1:8000/predict)? ¿Porqué?

Para probar que el método `POST` funciona, `FastAPI` nos habilita la ruta [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs) por defecto en donde podemos probar lo que hemos desarrollado en la API.

Probemos en vivo nuestra API REST!

<p align="center">
  <img src="https://media.tenor.com/ug1DBRF_MjIAAAAC/bill-oreilly-well-do-it-live.gif" width="400">
</p>

### Juntando todo

Ahora que ya conocemos como implementar tanto el Back End como el Front End, haremos que ambas partes trabajen de manera **simultánea**. Para lograr esto, seguiremos el siguiente flujo:

1. Usuario solicita una predicción de nuestro modelo a través del Front
2. Front recibe solicitud y la envia al Back
3. Back procesa y devuelve la predicción solicitada al Front
4. Front envia la predicción al Usuario

Considerando el esquema señalado, necesitamos entonces una **vía de comunicación entre el Front y el Back** para procesar la solicitud del usuario. 

Si recordamos que el Back es una API REST, podemos simplemente enviar un *request* al Back y así retornar la respuesta! 

Esto lo podemos hacer a través de la librería `requests`, veamos un ejemplo:

<p align="center">
  <img src="https://media1.tenor.com/m/of1uLME1mx8AAAAd/put-them-together-build.gif" width="400">
</p>

In [ ]:
import requests

# payload
data = {
    "sepal_length": 3.5,
    "sepal_width": 2.0,
    "petal_length": 1.6,
    "petal_width": 5.4
}

url = "http://127.0.0.1:8000/predict" # el back debe estar ejecutándose
response = requests.post(url, json = data) # invocamos un POST enviando la data en el payload

response.json()

Luego, podemos modificar nuestro script de `Gradio` para realizar requests al back. 

Primero definimos una función de predicción usando `requests`:

In [ ]:
import requests

# primero definimos una función para obtener respuestas del back a través de requests
def get_backend_prediction(
        sepal_length: float, 
        sepal_width: float, 
        petal_length: float, 
        petal_width: float,
        ):

    # payload
    data = {
        "sepal_length": sepal_length,
        "sepal_width": sepal_width,
        "petal_length": petal_length,
        "petal_width": petal_width,
    }

    url = "http://127.0.0.1:8000/predict" # url del back end
    response = requests.post(url, json = data) # código de respuesta
    label = response.json()["label"] # obtener contenido de la respuesta

    return label

Finalmente levantamos la misma aplicación de `gradio` que habíamos escrito antes, aunque ahora usando la función `get_backend_prediction`:

In [ ]:
import gradio as gr

# la única linea que cambia es la función a invocar
with gr.Blocks(theme = gr.themes.Base()) as demo:
    gr.Markdown(
    """
    # Iris ML Demo
    Bienvenid@ a Iris ML demo! Esta herramienta esta diseñada para predecir la clase de una flor a partir de sus características usando Machine Learning.
    ## Cómo usar este demo?
    Usar esta herramienta es fácil! Sólo debes seguir los siguientes pasos:
    1. Fijar los valores de **Sepal Length**, **Sepal Width**, **Petal Length** y **Petal Width**.
    2. Observar el tipo de flor que predice el modelo.
    
    Eso es todo! Estás list@ para explorar y predecir diferentes tipos de flores. Que lo disfrutes!
    """)

    with gr.Row():
        with gr.Column():
            sepal_length_slider = gr.Slider(label = 'Sepal Length', minimum = 0, maximum = 10, value = 3.8)
            sepal_width_slider = gr.Slider(label = 'Sepal Width', minimum = 0, maximum = 10, value = 6.4)
            petal_length_slider = gr.Slider(label = 'Petal Length', minimum = 0, maximum = 10, value = 7.3)
            petal_width_slider = gr.Slider(label = 'Petal Width', minimum = 0, maximum = 10, value = 4.9)

        with gr.Column():
            label = gr.Text(label = 'Predicted Label') # se define un nombre para la salida
    
    with gr.Row():
        button = gr.Button(value = 'Predict!')

    # setear interactividad
    inputs = [sepal_length_slider, sepal_width_slider, petal_length_slider, petal_width_slider]
    outputs = [label]
    button.click(fn = get_backend_prediction, inputs = inputs, outputs = outputs) # esta linea invoca el back end

    examples = [
        [0.5, 1.5, 2.5, 3.5], # example 1
        [1, 3, 5, 7], # example 2
    ]
    gr.Examples(examples = examples, inputs = inputs) 

    demo.launch(share = True)

Finalmente, deberían ser capaces de ejecutar ambos script (una terminal para el back, otra terminal para el front) y montar su aplicación fuera de jupyter notebook.